In [ ]:
import os
import os.path as path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import math

import imageio
import glob
from skimage import io

import albumentations as A
import json
import random

from numpy import savez_compressed
#import kornia as K

object_name = 'deisel_engine'
img_container = []
count=0

for filename in glob.glob('../all/*.jpg'):
    print(filename)
    head, tail = os.path.split(filename)
    filerename = head + "/resizedDataset/"+str(count)+'.jpg'
    print(filerename)
    im=io.imread(filename)
    im=cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (720,720), interpolation = cv2.INTER_AREA)
    cv2.imwrite(filerename, im)
    count +=1

After downloading the json format, put it in the images folder and make another folder for augmentation

In [ ]:
def augumentOneByOne(reiterate,outPath,idx,indexed,npAugumentedArray,ImageFolderPath,oldFileName,totalLengthOfSegmentPoints,transform,makePointsArr,image,makeLabelArray,keepTrackOfSegpoints,transformedIndex,totalLengthOfKeypoints,width,height,keepTrackOfLabels):
    newFileName = "aug-transformedIndex"+str(reiterate)+str(idx)+str(indexed)+str(transformedIndex)+oldFileName
    transformed = transform(image=image, keypoints=makePointsArr)
    transformed_image = transformed['image']
    transformed_keypoints = transformed['keypoints']
    transformed_keypoints = np.array(transformed_keypoints)
    ## Transformation ends here ##

    fileID = 0
    if totalLengthOfKeypoints != 0:
        keyPoints = transformed_keypoints[-totalLengthOfKeypoints:]
        transformed_keypoints = transformed_keypoints[:-totalLengthOfKeypoints]
        keyPointsLabels = makeLabelArray[-totalLengthOfKeypoints:]
        makeLabelArray = makeLabelArray[:-totalLengthOfKeypoints]

        AllKeypointsAppended = []
        for ind in range(0,len(keyPoints)):
            obj = {
                "x": keyPoints[ind][0] *100/width,
                "y": keyPoints[ind][1] * 100/height,
                "keypointlabels":keyPointsLabels[ind]
            }
            AllKeypointsAppended.append(obj)
    keepTrackOfIndex = 0

    appendPoints = []
    for ind in range(0,len(keepTrackOfSegpoints)):
        listOfPoints = transformed_keypoints[keepTrackOfIndex:keepTrackOfIndex+keepTrackOfSegpoints[ind]]
        listOfPoints[:,0] = listOfPoints[:,0] * 100/width
        listOfPoints[:,1] = listOfPoints[:,1] * 100/height
        keepTrackOfIndex = keepTrackOfIndex + keepTrackOfSegpoints[ind]
        obj = {
            "points":listOfPoints.tolist(),
            "polygonlabels":[keepTrackOfLabels[ind]],
            "original_width":width,
            "original_height":height
        }
        appendPoints.append(obj)
    
    if (totalLengthOfKeypoints != 0) and (totalLengthOfSegmentPoints != 0):
        finalImageJSON = {
            "image":newFileName,
            "id":fileID,
            "kp-1":AllKeypointsAppended,
            "label":appendPoints
        }
    elif (totalLengthOfSegmentPoints != 0):
        finalImageJSON = {
            "image":newFileName,
            "id":fileID,
            "label":appendPoints
        }
    else:
        finalImageJSON = {
            "image":newFileName,
            "id":fileID,
            "kp-1":AllKeypointsAppended
        }
    #transformed_image = cv2.cvtColor(transformed_image, cv2.COLOR_RGB2GRAY)
    #transformed_image = (transformed_image - np.min(transformed_image))/(np.max(transformed_image) - np.min(transformed_image))
    ## Save transformed Image
    #npAugumentedArray.append(transformed_image)
    cv2.imwrite(outPath+newFileName, transformed_image)
    #cv2.imwrite(outPath+newFileName, transformed_image)
    return finalImageJSON


def makeAugmentation(jsonFormat,ImageFolderPath,outPath,numberofTransformsPerImage):
    npAugumentedArray = []
    dataset_dicts = []
    finalJSONstructure = []
    for idx, v in enumerate(jsonFormat):
        if "annotator" in v:
            del v["annotator"]
            del v["annotation_id"]
            del v["created_at"]
            del v["updated_at"]
            del v["lead_time"]
      
        makePointsArr = []
        keepTrackOfSegpoints =[]
        makeLabelArray = []
        keepTrackOfLabels = []
        
        if "kp-1" in v:
            totalLengthOfKeypoints = len(v["kp-1"])
        else:
            totalLengthOfKeypoints = 0
        totalLengthOfSegmentPoints = 0
        if "label" in v:           
            for idxl, valLabel in enumerate(v["label"]):
                #if "original_height" in valLabel:
                #    del valLabel["original_height"]
                #    del valLabel["original_width"]

                totalLengthOfSegmentPoints += len(valLabel["points"])
                keepTrackOfSegpoints.append(len(valLabel["points"]))

                for idxpl, valPolygonLabel in enumerate(valLabel["polygonlabels"]):
                    keepTrackOfLabels.append(valPolygonLabel)
                for idxp, valPoints in enumerate(valLabel["points"]):
                    makePointsArr.append(valPoints)  ##First Append Segmentationpoints

            indexOfLabels = 0
            for pointsCount in keepTrackOfSegpoints:
                for i in range(0,pointsCount):
                    makeLabelArray.append(keepTrackOfLabels[indexOfLabels])   ## Append LABELS by iterating for each point in segmentation
                indexOfLabels += 1

        if totalLengthOfKeypoints != 0:
            
            for idxkp, valKP in enumerate(v["kp-1"]):
                if "width" in valKP:
                    del valKP["width"]
                    del valKP["original_width"]
                    del valKP["original_height"]
                makePointsArr.append([valKP["x"],valKP["y"]])  ## Second Append keypoints
                makeLabelArray.append(valKP["keypointlabels"])

        makePointsArr = np.array(makePointsArr)
        imagePath = ImageFolderPath+str(v["image"])
        #print("image done: ",imagePath)
        image = cv2.imread(imagePath)
        height, width = cv2.imread(imagePath).shape[:2]
        if len(makePointsArr) >=1 : 
            #print(width,height)
            makePointsArr[:,0] = makePointsArr[:,0] * width/100
            makePointsArr[:,1] = makePointsArr[:,1] * height/100
            
        elif len(makePointsArr) ==0:  ## this array is empty
            continue

        ## Transformation starts here ##
        
        transformedIndex = 0
        oldFileName = str(idx)+v["image"]
        v["image"] = oldFileName
        numberofTransforms = numberofTransformsPerImage  ## This indicates the number of transforms per image, Total 40
        reiterate=0
        for i in range(0,numberofTransforms):
            if i == 0:
                transform = A.Compose(
                    [A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))

            elif i == 1:
                transform = A.Compose(
                    [A.RandomBrightness (limit=0.2, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))


            elif i == 2:
                transform = A.Compose(
                    [A.Blur(blur_limit=3)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 3:
                transform = A.Compose(
                    [A.RandomBrightnessContrast(p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
                
            elif i == 4:
                transform = A.Compose(
                    [A.MedianBlur(blur_limit=3, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))

            elif i == 5:
                transform = A.Compose(
                    [A.InvertImg()],
                    keypoint_params=A.KeypointParams(format='xy'))
                
            elif i == 6:
                transform = A.Compose(
                    [A.RGBShift(r_shift_limit=40, g_shift_limit=60, b_shift_limit=90, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 7:
                transform = A.Compose(
                    [A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0, angle_upper=1, num_flare_circles_lower=6, num_flare_circles_upper=10, src_radius=400, src_color=(255, 255, 255), always_apply=False,p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            
            elif i == 8:
                transform = A.Compose(
                    [A.MotionBlur(p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            
            elif i == 9:
                transform = A.Compose(
                    [A.CLAHE()],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 10:
                transform = A.Compose(
                    [A.RGBShift(r_shift_limit=14, g_shift_limit=20, b_shift_limit=40, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
                
            elif i == 11:
                transform = A.Compose(
                    [A.HueSaturationValue(p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
                
            elif i == 12:
                transform = A.Compose(
                    [A.ChannelShuffle()],
                    keypoint_params=A.KeypointParams(format='xy'))
                
            elif i == 13:
                for reiterate in range(10,50,5):
                    transform = A.Compose(
                        [A.RGBShift(r_shift_limit=reiterate, g_shift_limit=reiterate, b_shift_limit=reiterate, p=1)],
                        keypoint_params=A.KeypointParams(format='xy'))
                    finalImageJSON = augumentOneByOne(reiterate,outPath,idx,i,npAugumentedArray,ImageFolderPath,oldFileName,totalLengthOfSegmentPoints,transform,makePointsArr,image,makeLabelArray,keepTrackOfSegpoints,transformedIndex,totalLengthOfKeypoints,width,height,keepTrackOfLabels)
                    transformedIndex += 1

                    finalJSONstructure.append(finalImageJSON)

            
            elif i == 14:
                transform = A.Compose(
                    [A.RandomToneCurve(p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 15:
                transform = A.Compose(
                    [A.JpegCompression (quality_lower=99, quality_upper=100, always_apply=False, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))

            elif i == 16:
                transform = A.Compose(
                    [A.RandomSnow(snow_point_lower=0.1, snow_point_upper=0.3, brightness_coeff=2.5,p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 17:
                transform = A.Compose(
                    [A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 18:
                transform = A.Compose(
                    [A.RandomRain (slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, drop_color=(200, 200, 200), blur_value=7, brightness_coefficient=0.7, rain_type=None, always_apply=False, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 19:
                for reiterate in range(10,150,10):
                    transform = A.Compose(
                        [A.RandomGamma(gamma_limit=(reiterate, reiterate+30), eps=None, p=1)],
                        keypoint_params=A.KeypointParams(format='xy'))
                    finalImageJSON = augumentOneByOne(reiterate,outPath,idx,i,npAugumentedArray,ImageFolderPath,oldFileName,totalLengthOfSegmentPoints,transform,makePointsArr,image,makeLabelArray,keepTrackOfSegpoints,transformedIndex,totalLengthOfKeypoints,width,height,keepTrackOfLabels)
                    transformedIndex += 1

                    finalJSONstructure.append(finalImageJSON)

                
            elif i == 20:
                transform = A.Compose(
                    [A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=1, alpha_coef=0.08, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 21:
                transform = A.Compose(
                    [A.Posterize (num_bits=4, always_apply=False, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 22:
                transform = A.Compose(
                    [A.Solarize(threshold=128,p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 23:
                for reiterate in range(10,150,10):
                    param = reiterate/100
                    transform = A.Compose(
                        [A.MultiplicativeNoise (multiplier=(param, param+0.2), per_channel=False, elementwise=False, p=1)],
                        keypoint_params=A.KeypointParams(format='xy'))
                    finalImageJSON = augumentOneByOne(reiterate,outPath,idx,i,npAugumentedArray,ImageFolderPath,oldFileName,totalLengthOfSegmentPoints,transform,makePointsArr,image,makeLabelArray,keepTrackOfSegpoints,transformedIndex,totalLengthOfKeypoints,width,height,keepTrackOfLabels)
                    transformedIndex += 1

                    finalJSONstructure.append(finalImageJSON)

            
            elif i == 24:
                for reiterate in range(1,10):
                    param = reiterate/100
                    transform = A.Compose(
                        [A.ISONoise (color_shift=(param, param+0.03), intensity=(0.1, 0.5), p=1)],
                        keypoint_params=A.KeypointParams(format='xy'))
                    finalImageJSON = augumentOneByOne(reiterate,outPath,idx,i,npAugumentedArray,ImageFolderPath,oldFileName,totalLengthOfSegmentPoints,transform,makePointsArr,image,makeLabelArray,keepTrackOfSegpoints,transformedIndex,totalLengthOfKeypoints,width,height,keepTrackOfLabels)
                    transformedIndex += 1

                    finalJSONstructure.append(finalImageJSON)

                
            #elif i == 25:
            #    transform = A.Compose(
            #        [A.Transpose()],
            #        keypoint_params=A.KeypointParams(format='xy'))
                
            elif i == 26:
                transform = A.Compose(
                    [A.GaussianBlur (blur_limit=(3, 7), sigma_limit=0, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 27:
                for reiterate in range(1,9):
                    param = reiterate/10
                    transform = A.Compose(
                        [A.ColorJitter(brightness=param, contrast=param, saturation=param, hue=param, p=1)],
                        keypoint_params=A.KeypointParams(format='xy'))
                    finalImageJSON = augumentOneByOne(reiterate,outPath,idx,i,npAugumentedArray,ImageFolderPath,oldFileName,totalLengthOfSegmentPoints,transform,makePointsArr,image,makeLabelArray,keepTrackOfSegpoints,transformedIndex,totalLengthOfKeypoints,width,height,keepTrackOfLabels)
                    transformedIndex += 1

                    finalJSONstructure.append(finalImageJSON)
            
            elif i == 28:
                transform = A.Compose(
                    [A.Superpixels (p_replace=0.1, n_segments=100, max_size=128, interpolation=1,p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 29:
                transform = A.Compose(
                    [A.ToSepia(p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 30:
                transform = A.Compose(
                    [A.Emboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 31:
                transform = A.Compose(
                    [A.Equalize (mode='cv', by_channels=True, mask=None, mask_params=(),p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 32:
                transform = A.Compose(
                    [A.ISONoise (color_shift=(0.03, 0.08), intensity=(0.15, 0.7), p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
                
            elif i == 33:
                transform = A.Compose(
                    [A.ToSepia(p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 34:
                transform = A.Compose(
                    [A.VerticalFlip(p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))

                
            elif i == 35:
                transform = A.Compose(
                    [A.HorizontalFlip(p=1)],
                     #A.Resize(height=500, width=500, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
                
            elif i == 36:
                transform = A.Compose(
                    [A.ColorJitter(brightness=0.4, contrast=0.6, saturation=0.4, hue=0.6, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
                
            elif i == 37:
                transform = A.Compose(
                    [A.Solarize(threshold=64,p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
            
            elif i == 38:
                transform = A.Compose(
                    [A.MedianBlur(blur_limit=5, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
                
            elif i == 39:
                transform = A.Compose(
                    [A.RandomGamma(gamma_limit=(100, 160), eps=None, p=1)],
                    keypoint_params=A.KeypointParams(format='xy'))
                
            finalImageJSON = augumentOneByOne(reiterate,outPath,idx,i,npAugumentedArray,ImageFolderPath,oldFileName,totalLengthOfSegmentPoints,transform,makePointsArr,image,makeLabelArray,keepTrackOfSegpoints,transformedIndex,totalLengthOfKeypoints,width,height,keepTrackOfLabels)
            transformedIndex += 1
            
            finalJSONstructure.append(finalImageJSON)
        
        ## Save Real Image
        #npAugumentedArray.append(image)
        #print(idx)
        
        #cv2.imwrite(outPath+oldFileName, image)
        ## code to form a new JSON starts here ##
        #finalJSONstructure.append(v)
        
        del(makePointsArr)
        del(keepTrackOfSegpoints)
        del(makeLabelArray)
        del(keepTrackOfLabels)

    
    return finalJSONstructure,npAugumentedArray

KEYPOINT_COLOR = (0, 255, 0) # Green

For Level 0

In [ ]:
OutputFolder = "../Dataset/"    #Argument 1
directory = "augmentations"
path = os.path.join(OutputFolder, directory)
os.mkdir(path)

OutputFolder = path + "/"
directory = "Level0"
path_0 = os.path.join(OutputFolder, directory)
os.mkdir(path_0)

In [ ]:
json_file = "../Dataset/miniJSON/AnnotatedJSON.json"   #Argument 2
ImageFolderPath = "../Dataset/baseImages/"    #Argument 3
outPath = path_0 + "/"
numberofTransformsPerImage = 40 #from 1 to 40    #Argument 4

with open(json_file) as f:
    imgs_anns = json.load(f)
    
for idx, valKP in enumerate(imgs_anns):     # To change the file names in miniJSON; remove if the filename path in your JSON is correct
    head, tail = os.path.split(valKP["image"])
    
    tail = tail.split("-")     #Use this if you have the same images which you uploaded on LabelStudio. If you do not have the images, take images from Label studio coco format (In that case do not use this line).
    valKP["image"] = tail[1]

finalJSONstructure,npAugumentedArray = makeAugmentation(imgs_anns,ImageFolderPath,outPath,numberofTransformsPerImage)

fixIDs = 0
for idx, v in enumerate(finalJSONstructure):
    v["id"] = fixIDs
    fixIDs += 1
print("Number of images inside JSON",fixIDs)

#npAugumentedArray = np.array(npAugumentedArray)
#print(len(npAugumentedArray))
#print(npAugumentedArray.shape)

#savez_compressed(ImageFolderPath + "AugImages/augumentedDATA", npAugumentedArray)
#print("NPZ file saved!")

with open(outPath+"/augumentedJSON.json", "w") as outfile:
    json.dump(finalJSONstructure, outfile)

print("JSON file saved!")
print("Successful!")

For Level 1

In [ ]:
OutputFolder = path + "/"
directory = "Level1"
path_1 = os.path.join(OutputFolder, directory)
os.mkdir(path_1)

In [ ]:
json_file = path_0 + "/augumentedJSON.json"
ImageFolderPath = path_0 + "/"
outPath = path_1 + "/"
numberofTransformsPerImage = 20 #from 1 to 40

with open(json_file) as f:
    imgs_anns = json.load(f)

finalJSONstructure,npAugumentedArray = makeAugmentation(imgs_anns,ImageFolderPath,outPath,numberofTransformsPerImage)

fixIDs = 0
for idx, v in enumerate(finalJSONstructure):
    v["id"] = fixIDs
    fixIDs += 1
print("Number of images inside JSON",fixIDs)

#npAugumentedArray = np.array(npAugumentedArray)
#print(len(npAugumentedArray))
#print(npAugumentedArray.shape)

#savez_compressed(ImageFolderPath + "AugImages/augumentedDATA", npAugumentedArray)
#print("NPZ file saved!")

with open(outPath+"/augumentedJSON.json", "w") as outfile:
    json.dump(finalJSONstructure, outfile)

print("JSON file saved!")
print("Successful!")

For Level 2

In [ ]:
OutputFolder = path_1 + "/"
directory = "Level2"
path_2 = os.path.join(OutputFolder, directory)
os.mkdir(path_2)

In [ ]:
json_file = path_1 + "/augumentedJSON.json"
ImageFolderPath = path_1 + "/"
outPath = path_2 + "/"
numberofTransformsPerImage = 3 #from 1 to 40

with open(json_file) as f:
    imgs_anns = json.load(f)

finalJSONstructure,npAugumentedArray = makeAugmentation(imgs_anns,ImageFolderPath,outPath,numberofTransformsPerImage)

fixIDs = 0
for idx, v in enumerate(finalJSONstructure):
    v["id"] = fixIDs
    fixIDs += 1
print("Number of images inside JSON",fixIDs)

#npAugumentedArray = np.array(npAugumentedArray)
#print(len(npAugumentedArray))
#print(npAugumentedArray.shape)

#savez_compressed(ImageFolderPath + "AugImages/augumentedDATA", npAugumentedArray)
#print("NPZ file saved!")

with open(outPath+"/augumentedJSON.json", "w") as outfile:
    json.dump(finalJSONstructure, outfile)

print("JSON file saved!")
print("Successful!")